In [2]:
import requests 
import json
import pandas as pd 
from tqdm import tqdm

In [3]:
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36"}
urlItemSpidId = 'https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&include=advertisement&aggregations=2&trackity_id=26c3f858-66ce-cd5d-5950-836d257727ac&category=1795&page={}&urlKey=dien-thoai-smartphone'
urlItemDetail = 'https://tiki.vn/api/v2/products/{}?platform=web&spid={}'

In [15]:
def getItemId(data):
    id = data['id']
    spid = data['seller_product_id']
    return {
        'id':id,
        'spid': spid
    }
df1 = []

In [ ]:

for x in tqdm(range(0,10,1)):
    data = requests.get(urlItemSpidId.format(x), headers=headers).content
    data = json.loads(data)['data']
    res = list(map(getItemId,data))
    # print(res)
    df1 += res
itemIdSpid = pd.DataFrame(df1)
print(itemIdSpid)

In [19]:
def getAttributes(data):
    attributes = data['attributes']
    return {
        'attributes':attributes,
    }

def getDetails(rs):
    code = rs['code']
    value = rs['value']
    return {
      'code':code,
      'value': value
    }
df2 = []
for x in tqdm(range(itemIdSpid.shape[0])):
  a,b = itemIdSpid.values[x]
  data = requests.get(urlItemDetail.format(a,b), headers=headers).content
  data = json.loads(data)['specifications']
  rs = list(map(getAttributes, data))[0]['attributes']
  res = list(map(getDetails, rs))
  # print(res)
  df2 += res
attributes = pd.DataFrame(df2).set_index('code').T
print(attributes)


100%|██████████| 446/446 [02:12<00:00,  3.38it/s]


code  brand battery_capacity   bluetooth   brand brand_country camera_sau  \
value  Vivo       \t5000 mAh  \tA2DP, LE  Xiaomi    Trung Quốc    \t13 MP   

code  camera_truoc  cart_slot     chip_do_hoa         cpu_speed  ... xem_phim  \
value       \t5 MP  \tMicroSD  PowerVR GE8320  \t8 nhân 2.0 GHz  ...       Có   

code  brand display_type item_model_number jack_headphone khe_sim loai_sim  \
value   ZTE       AMOLED          V40 Vita          Không       2     nano   

code  port_sac                     resolution screen_size  
value   Type C  Full HD+ (1080 x 2160 pixels)    6.7 inch  

[1 rows x 7830 columns]


In [21]:
attributes2 = pd.DataFrame(df2)
print(attributes2)

                  code                          value
0                brand                           Vivo
1     battery_capacity                     \t5000 mAh
2            bluetooth                     \tA2DP, LE
3                brand                         Xiaomi
4        brand_country                     Trung Quốc
...                ...                            ...
7825           khe_sim                              2
7826          loai_sim                           nano
7827          port_sac                         Type C
7828        resolution  Full HD+ (1080 x 2160 pixels)
7829       screen_size                       6.7 inch

[7830 rows x 2 columns]


In [ ]:
# attributesList = pd.DataFrame(res).set_index('code').T
# print(attributesList)

In [ ]:
# attributesList.to_json()

In [22]:
attributes2.to_csv('itemDetail.csv', index=False)